# Multivariate Gaussian Head PatchTST

In [1]:
import matplotlib.pyplot as plt
from utils import *
import os
os.chdir('/home/kreffert/Probabilistic_LTSF/BasicTS/')
from prob.prob_head import ProbabilisticHead
path= '6f0ff289427f26b39444d02254fe169c'#'27d40957374f98d2d16be997f2c114de'
runner = init(model='PatchTST', dataset='ETTh1_prob', path=path, no_logging=False)

2025-03-28 14:34:47,508 - easytorch-launcher - INFO - Initializing runner "<class 'basicts.runners.runner_zoo.simple_prob_tsf_runner.SimpleProbTimeSeriesForecastingRunner'>"
2025-03-28 14:34:47,509 - easytorch-env - INFO - Disable TF32 mode
2025-03-28 14:34:47,510 - easytorch - INFO - Set ckpt save dir: 'checkpoints/m_gaussian_PatchTST/ETTh1_100_336_96/e75adfdf06ccf0bd48ba53933d5da2ca'
2025-03-28 14:34:47,510 - easytorch - INFO - Building model.


DESCRIPTION: An Example Config
GPU_NUM: 1
RUNNER: <class 'basicts.runners.runner_zoo.simple_prob_tsf_runner.SimpleProbTimeSeriesForecastingRunner'>
DATASET:
  NAME: ETTh1
  TYPE: <class 'basicts.data.simple_tsf_dataset.TimeSeriesForecastingDataset'>
  PARAM:
    dataset_name: ETTh1
    train_val_test_ratio: [0.6, 0.2, 0.2]
    input_len: 336
    output_len: 96
SCALER:
  TYPE: <class 'basicts.scaler.z_score_scaler.ZScoreScaler'>
  PARAM:
    dataset_name: ETTh1
    train_ratio: 0.6
    norm_each_channel: True
    rescale: False
MODEL:
  NAME: PatchTST
  ARCH: <class 'baselines.PatchTST.arch.patchtst_arch.PatchTST'>
  PARAM:
    enc_in: 7
    seq_len: 336
    pred_len: 96
    e_layers: 3
    n_heads: 4
    d_model: 16
    d_ff: 128
    dropout: 0.3
    fc_dropout: 0.3
    head_dropout: 0.0
    patch_len: 16
    stride: 8
    individual: 1
    padding_patch: end
    revin: 0
    affine: 0
    subtract_last: 0
    decomposition: 0
    kernel_size: 25
    head_type: probabilistic
    distri

2025-03-28 14:34:48,302 - easytorch-training - INFO - Initializing training.
2025-03-28 14:34:48,303 - easytorch-training - INFO - Set clip grad, param: {'max_norm': 5.0}
2025-03-28 14:34:48,303 - easytorch-training - INFO - Building training data loader.
2025-03-28 14:34:48,306 - easytorch-training - INFO - Train dataset length: 8209


PatchTST


2025-03-28 14:34:49,666 - easytorch-training - INFO - Set optim: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0002
    maximize: False
    weight_decay: 0.0001
)
2025-03-28 14:34:49,666 - easytorch-training - INFO - Set lr_scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x7f3d79d8acd0>
2025-03-28 14:34:49,667 - easytorch-training - INFO - Load model from : /home/kreffert/Probabilistic_LTSF/BasicTS/checkpoints/m_gaussian_PatchTST/ETTh1_100_336_96/6f0ff289427f26b39444d02254fe169c
2025-03-28 14:34:49,669 - easytorch-training - INFO - Loading Checkpoint from '/home/kreffert/Probabilistic_LTSF/BasicTS/checkpoints/m_gaussian_PatchTST/ETTh1_100_336_96/6f0ff289427f26b39444d02254fe169c/PatchTST_008.pt'
2025-03-28 14:34:50,729 - easytorch-training - INFO - Resume training
2025-03-28 14:34:50,732 - easytorch-training - INFO 

In [2]:
def get_predictions_1(runner, data_loader="test", num_samples=0):
    if data_loader == "test":
        dataloader = runner.test_data_loader
    elif data_loader == "val":
        dataloader = runner.val_data_loader
    else:
        dataloader = runner.train_data_loader
    model = runner.model
    model.eval()  # Ensure model is in eval mode
    device = 'cuda:0'
    model.to(device)  # Move to appropriate device
    predictions = []
    past_actuals = []
    future_actuals = []
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader)):
            history = batch['inputs'][..., :1]     # B, L, N -> i.e. ignore the remaining features
            targets = batch['target'][..., :1]
            
            # TODO scaler
            model_return = runner.forward(batch, epoch=100, iter_num=100, train=False)
            if num_samples>0:
                prob_head = ProbabilisticHead(1, 1, runner.distribution_type)
                forecasts = prob_head.sample(model_return['prediction'], num_samples=num_samples) # [samples x bs x seq_len x nvars]
                forecasts = forecasts.permute(1, 0, 2, 3)       # [bs x samples x seq_len x nvars]
            else:
                forecasts = model_return['prediction']
            #forecasts = model(batch['inputs'][..., :1].to(device), batch['target'].to(device), batch_seen=0, epoch=0, train=False)
            history = model_return['inputs']
            targets = model_return['target']
            
            history, targets, forecasts = history.to('cpu'), targets.to('cpu'), forecasts.to('cpu') 
            future_actuals.append(targets)
            past_actuals.append(history)
            predictions.append(forecasts)
    
    # Convert to numpy if needed
    predictions = torch.cat(predictions, dim=0).numpy()
    past_actuals = torch.cat(past_actuals, dim=0).numpy()
    future_actuals = torch.cat(future_actuals, dim=0).numpy()
    return predictions, past_actuals, future_actuals

print(runner.quantiles)
runner.quantiles = [0.0001, 0.5, 0.9999]
print(runner.quantiles)
predictions, past_actuals, future_actuals = get_predictions_1(runner, data_loader="test", num_samples=10)

[]
[0.0001, 0.5, 0.9999]


39it [00:03, 12.89it/s]


In [4]:
print(predictions.shape)
#print(past_actuals.shape)
print(future_actuals.shape)
import scoringrules as sr

print(sr.energy_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
).shape) 
print(np.mean(sr.energy_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
    p=1.0
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
    p=0.5
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
    p=2.0
)))

print(sr.energy_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
).shape) 
print(np.mean(sr.energy_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
    p=1.0
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
    p=0.5
)))


print(future_actuals.squeeze().transpose(0, 2, 1).shape)
print(predictions.transpose(0, 3, 1, 2).shape)

print(np.mean(sr.variogram_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
    p=2.0
)))

(2449, 10, 96, 7)
(2449, 96, 7, 1)
(2449, 96)
1.3277494
27.374022
6.310543
439.1061
(2449, 7)
4.9782248
5610.2925
1396.326
(2449, 7, 96)
(2449, 7, 10, 96)
94456.21


In [44]:
obs = np.random.randn(100)
fct = obs[:,None] + np.random.randn(100, 21) * 0.1
print(obs.shape)
print(fct.shape)
print(sr.crps_ensemble(obs, fct))

print(np.mean(sr.crps_ensemble(
    future_actuals.squeeze(),
    predictions.transpose(0, 2, 3, 1),  # [bs x seq_len x nvars x samples]
    axis=-1,)))

(100,)
(100, 21)
[0.02457977 0.0200618  0.02652772 0.01867211 0.01765181 0.02418842
 0.01344769 0.02396534 0.0289362  0.01808945 0.02633872 0.02049001
 0.0298497  0.02356089 0.02742191 0.0246078  0.01877053 0.02303512
 0.01886611 0.01242876 0.02403    0.02431674 0.02690632 0.02582324
 0.02853086 0.02242741 0.02238501 0.01767492 0.03500149 0.0279781
 0.02766123 0.02710787 0.0221866  0.02525498 0.02462118 0.02106466
 0.02085038 0.02306307 0.02492561 0.01818131 0.01383276 0.02959939
 0.02335296 0.02731497 0.02023336 0.02222607 0.02331692 0.01749067
 0.02231545 0.0257172  0.03331453 0.01819047 0.02011172 0.02362504
 0.0235801  0.02108001 0.02247826 0.02006906 0.01263671 0.02531506
 0.03254549 0.01601181 0.02191153 0.03119792 0.02888505 0.02397813
 0.01813934 0.02476408 0.02345666 0.01789013 0.01927925 0.03515691
 0.02163801 0.02216832 0.01455214 0.01439254 0.0189784  0.02981418
 0.02940791 0.02866944 0.01524208 0.01221056 0.0221438  0.02212867
 0.01918489 0.02055038 0.01981398 0.02981398 0

In [41]:
print(predictions.shape)
print(past_actuals.shape)
print(future_actuals.shape)
import scoringrules as sr

print(sr.energy_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
).shape) 
print(np.mean(sr.energy_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
    p=1.0
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
    p=0.5
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze(), # [bs x seq_len x nvars x samples]
    predictions.transpose(0, 2, 1, 3),  # [bs x seq_len x samples x nvars]
    p=2.0
)))

print(sr.energy_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
).shape) 
print(np.mean(sr.energy_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
    p=1.0
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
    p=0.5
)))
print(np.mean(sr.variogram_score(
    future_actuals.squeeze().transpose(0, 2, 1), # [bs x nvars x seq_len]
    predictions.transpose(0, 3, 1, 2),  # [bs  x nvars x samples x seq_len]
    p=2.0
)))

(2449, 10, 96, 7)
(2449, 336, 7, 1)
(2449, 96, 7, 1)
(2449, 96)
1.3274951
27.375677
6.3103676
438.97415
(2449, 7)
4.977914
5609.5693
1396.0065
94445.13


In [14]:
import seaborn as sns
def plot_predictions(predictions, window, serie, ax, ci=None, quantile_levels=[]):
    if ci != None:
        if len(quantile_levels)>1: # quantile forecast
            ci = [i / 100 if i > 1 else i for i in ci]
            assert ci[0] in quantile_levels
            assert ci[1]in quantile_levels
            assert 0.5 in quantile_levels 
            lower_bound = predictions[window, :, serie, quantile_levels.index(ci[0])]
            upper_bound = predictions[window, :, serie, quantile_levels.index(ci[1])]
            median_pred = predictions[window, :, serie, quantile_levels.index(0.5)]
        else:# non quantile prob forecast -> need to calculate the quantiles first
            # Extract mean and std
            median_pred = predictions[window, :, serie, 0]  # Shape: [num_time_steps]
            std = predictions[window, :, serie, 1]   # Shape: [num_time_steps]
            
            # Create a grid of x values
            #x = range(predictions.shape[1])
            # Compute the PDFs for all time steps at once
            #pdfs = norm.pdf(x, loc=median_pred, scale=std)  # Shape: [num_points, num_time_steps]
            
            #ax.plot(x, pdfs)#label=[f"Time Step {t+1} (μ={mean[t]}, σ={std[t]})" for t in range(predictions.shape[0])])
            
            lower_bound = median_pred + 2*std #np.percentile(predictions[window, :, :, serie], ci[0], axis=0)
            upper_bound = median_pred - 2*std #np.percentile(predictions[window, :, :, serie], ci[1], axis=0)
            #median_pred = np.percentile(predictions[window, :, :, serie], 50, axis=0)

        pred_range = range(len(past_actuals[window]), len(past_actuals[window]) + len(median_pred))
        ax.fill_between(pred_range, lower_bound, upper_bound, color='red', alpha=0.3, label=f"{ci[0]}-{ci[1]}% CI")
        ax.plot(pred_range, median_pred, color='red', linestyle='dashed', label="Median Prediction")
    else:# point or sample forecasts
        if len(predictions.squeeze().shape) >3:
            batches, num_samples, pred_length, num_series = predictions.shape
            palette = sns.color_palette("deep", n_colors=7)[1:]  # Change the number to match your series count
            line_styles = ['-', '--', '-.', ':', '-', '--', '-.']  # Different line styles
            num_samples = 2
            for i in range(num_samples):
                ax.plot(range(len(past_actuals[window]), len(past_actuals[window]) + pred_length),
                    predictions[window, i, :, serie], color=palette[i % len(palette)], linestyle=line_styles[i % len(line_styles)], label=f'Series {i+1}', alpha=0.7)
        else:
            ax.plot(range(len(past_actuals[window]), len(past_actuals[window]) + len(predictions[window])),
                predictions[window, :, serie], label="Prediction", color='orange', linestyle='dashed')



def plot_time_series_1(past_actuals, future_actuals, predictions, windows, series, ci=[5, 95], quantile_levels=[]):
    num_rows = len(series)  # One row per feature
    num_cols = len(windows)  # One column per time series index

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 3), sharex=True, sharey=False)

    if num_rows == 1 or num_cols == 1:
        axes = np.array(axes).reshape(num_rows, num_cols)  # Ensure 2D indexing

    for row, serie in enumerate(series):
        for col, window in enumerate(windows):
            ax = axes[row, col]  # Get the correct subplot

            # Plot past actuals
            ax.plot(range(len(past_actuals[window])), past_actuals[window, :, serie], label="Past", color='blue')

            # Plot future actuals
            ax.plot(range(len(past_actuals[window]), len(past_actuals[window]) + len(future_actuals[window])),
                    future_actuals[window, :, serie], label="Future", color='red')
            
            plot_predictions(predictions, window, serie, ax, ci=ci, quantile_levels=quantile_levels)
            
            ax.axvline(x=len(past_actuals[window]), color='black', linestyle='dotted')  # Mark forecast start
            if row == 0:
                ax.set_title(f"Window {window}")
            if col == 0:
                ax.set_ylabel(f"Time Series {serie}")
            ax.legend(fontsize=6)
    plt.tight_layout()
    plt.show()

windows = [0, 5, 10, 16]
series = [0, 1, 2, 3, 4, 5, 6]
# plot_time_series(past_actuals, future_actuals, predictions, windows, series, ci=(0.05, 0.95), quantile_levels=[])
plot_time_series_1(past_actuals, future_actuals, predictions, windows, series, ci=None, quantile_levels=runner.quantiles)

In [ ]:
def plot_time_series_1(past_actuals, future_actuals, predictions, windows, series, ci=[5, 95], quantile_levels=[]):
    num_rows = len(series)  # One row per feature
    num_cols = len(windows)  # One column per time series index

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 3), sharex=True, sharey=False)

    if num_rows == 1 or num_cols == 1:
        axes = np.array(axes).reshape(num_rows, num_cols)  # Ensure 2D indexing

    for row, serie in enumerate(series):
        for col, window in enumerate(windows):
            ax = axes[row, col]  # Get the correct subplot

            # Plot past actuals
            ax.plot(range(len(past_actuals[window])), past_actuals[window, :, serie], label="Past", color='blue')

            # Plot future actuals
            ax.plot(range(len(past_actuals[window]), len(past_actuals[window]) + len(future_actuals[window])),
                    future_actuals[window, :, serie], label="Future", color='green')

            if ci != None:
                if len(quantile_levels)>1:# quantile forecast
                    ci = [i / 100 if i > 1 else i for i in ci]
                    assert ci[0] in quantile_levels
                    assert ci[1]in quantile_levels
                    assert 0.5 in quantile_levels 
                    lower_bound = predictions[window, :, serie, quantile_levels.index(ci[0])]
                    upper_bound = predictions[window, :, serie, quantile_levels.index(ci[1])]
                    median_pred = predictions[window, :, serie, quantile_levels.index(0.5)]
                else:# non quantile prob forecast -> need to calculate the quantiles first
                    # Extract mean and std
                    median_pred = predictions[window, :, serie, 0]  # Shape: [num_time_steps]
                    std = predictions[window, :, serie, 1]   # Shape: [num_time_steps]
                    
                    # Create a grid of x values
                    #x = range(predictions.shape[1])
                    # Compute the PDFs for all time steps at once
                    #pdfs = norm.pdf(x, loc=median_pred, scale=std)  # Shape: [num_points, num_time_steps]
                    
                    #ax.plot(x, pdfs)#label=[f"Time Step {t+1} (μ={mean[t]}, σ={std[t]})" for t in range(predictions.shape[0])])
                    
                    lower_bound = median_pred + 2*std #np.percentile(predictions[window, :, :, serie], ci[0], axis=0)
                    upper_bound = median_pred - 2*std #np.percentile(predictions[window, :, :, serie], ci[1], axis=0)
                    #median_pred = np.percentile(predictions[window, :, :, serie], 50, axis=0)
    
                pred_range = range(len(past_actuals[window]), len(past_actuals[window]) + len(median_pred))
                ax.fill_between(pred_range, lower_bound, upper_bound, color='red', alpha=0.3, label=f"{ci[0]}-{ci[1]}% CI")
                ax.plot(pred_range, median_pred, color='red', linestyle='dashed', label="Median Prediction")
            else:# point forecasts
                ax.plot(range(len(past_actuals[window]), len(past_actuals[window]) + len(predictions[window])),
                    predictions[window, :, serie], label="Prediction", color='red', linestyle='dashed')
            
            ax.axvline(x=len(past_actuals[window]), color='black', linestyle='dotted')  # Mark forecast start
            if row == 0:
                ax.set_title(f"Window {window}")
            if col == 0:
                ax.set_ylabel(f"Time Series {serie}")
            ax.legend(fontsize=6)
    plt.tight_layout()
    plt.show()

print(runner.quantiles)
runner.quantiles = [0.0001, 0.5, 0.9999]
print(runner.quantiles)
predictions, past_actuals, future_actuals = get_predictions(runner, data_loader="test")

windows = [0, 5, 10, 16]
series = [0, 1, 2, 3, 4, 5, 6]
# plot_time_series(past_actuals, future_actuals, predictions, windows, series, ci=(0.05, 0.95), quantile_levels=[])
plot_time_series_1(past_actuals, future_actuals, predictions, windows, series, ci=(0.0001, 0.9999), quantile_levels=runner.quantiles)

In [ ]:
import torch
from torch.optim import SGD
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models import SingleTaskGP
import gpytorch
from gpytorch.constraints import GreaterThan
from gpytorch.distributions import MultivariateNormal
import matplotlib.pyplot as plt
import seaborn as sns

class SeriesSampler_1:
    def __init__(self, gp_model, runner, num_samples=1):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.gp_model = gp_model
        self.runner = runner
        self.base_model = runner.model 
        self.test_loader = runner.test_data_loader
        self.val_loader = runner.val_data_loader
        self.train_loader = runner.train_data_loader
        self.num_samples = num_samples

    def run(self):
        # 1. Fit the GP
        self.fit_gp()

        # 2. Make the predictions
        self.predictions = self.get_predictions_base_model(on='test')
        window = 0
        serie = 0
        mean_pred = self.predictions[window, :, serie, 0]  # Shape: [num_time_steps]
        std_pred = self.predictions[window, :, serie, 1]   # Shape: [num_time_steps]
        
        # 4. define new posterior
        self.new_posterior = self.init_distribution(mean_pred, std_pred)

        return self.sample(self.new_posterior)

    def get_batch(self, data_loader, first=True, window=0, series=0):
        if first:
            # select the first batch
            _iter = iter(data_loader) 
            batch = next(_iter)
        else:
            # select the last batch
            num_batches = len(data_loader)
            if num_batches == 0:
                return None, None
            _iter = iter(data_loader)
            for i in range(num_batches - 1):
                next(_iter)
            batch = next(_iter)

        history = batch['inputs'][..., :1]     # bs x length x num_series x features [33, 336, 7, 1]
        target = batch['target'][..., :1]     # bs x length x num_series x features [33, 336, 7, 1]
        targets = torch.cat([history, target], dim=1)  # batch_size, horizon, num_series = (32, 360, 7)
        targets = targets[window, :, series]  # only select first batch/window at first (360, num_series)
        horizon_length = targets.shape[0]
    
        x = torch.arange(horizon_length).to(self.device, dtype=torch.float32).unsqueeze(-1)  # [360, 1]
        y = targets.to(self.device, dtype=torch.float32)  # [360, num_series, 1]
        return x, y

    def fit_gp(self):
        x, y = self.get_batch(self.train_loader, first=False)
        self.gp_train_x, self.gp_train_y = x, y
        self.gp_model = self.gp_model(train_X=x, train_Y=y, 
                                      mean_module=gpytorch.means.ConstantMean())  # Disable automatic output standardization)
        
        # Register noise constraint
        self.gp_model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))
        
        mll = ExactMarginalLogLikelihood(likelihood=self.gp_model.likelihood, model=self.gp_model)
        # Set mll and all submodules to the specified dtype and device
        mll = mll.to(x.device)
    
        optimizer = SGD([{"params": self.gp_model.parameters()}], lr=0.025)
        NUM_EPOCHS = 600
    
        self.gp_model.train()
    
        for epoch in range(NUM_EPOCHS):
            optimizer.zero_grad()
            output = self.gp_model(x)
            loss = -mll(output, self.gp_model.train_targets)
            loss.backward()
            if (epoch + 1) % 100 == 0:
                print(
                    f"Epoch {epoch+1:>3}/{NUM_EPOCHS} - Loss: {loss.item():>4.3f} "
                    f"lengthscale: {self.gp_model.covar_module.lengthscale.item():>4.3f} "
                    f"noise: {self.gp_model.likelihood.noise.item():>4.3f}"
                )
            optimizer.step()

    def get_predictions_base_model(self, on='val'):
        self.base_model.eval()  # Ensure model is in eval mode
        self.base_model.to(self.device)  # Move to appropriate device
        predictions = []
        if on == 'val':
            dataloader = self.val_loader
        else: 
            dataloader = self.test_loader
        with torch.no_grad():
            for batch_idx, batch in tqdm(enumerate(dataloader)):
                # TODO scaler 
                model_return = self.runner.forward(batch, epoch=None, iter_num=None, train=False)
                if self.runner.scaler:
                    model_return['prediction'] = self.runner.scaler.inverse_transform(model_return['prediction'])
                    model_return['target'] = self.runner.scaler.inverse_transform(model_return['target'])
                    model_return['inputs'] = self.runner.scaler.inverse_transform(model_return['inputs'])
                #model_return = self.runner.postprocessing(model_return)
                #forecasts = model(batch['inputs'][..., :1].to(device), batch['target'].to(device), batch_seen=0, epoch=0, train=False)
                forecasts = model_return['prediction']
                predictions.append(forecasts)
        predictions = torch.cat(predictions, dim=0)
        return predictions

    def init_distribution(self, mean_pred, std_pred):
        x, y = self.get_batch(self.test_loader, first=True)
        
        self.gp_model.eval()  # Ensure model is in evaluation mode
        
        # Filter x to match the length of mean_pred
        x_filter = x[-len(mean_pred):]
        
        # 1. Compute the kernel covariance matrix for the filtered test points
        kernel_matrix = self.gp_model.covar_module(x_filter).evaluate()
        
        # 2. Adjust the covariance matrix using the standard deviation predictions
        updated_cov = kernel_matrix + torch.diag(std_pred**2)  # Convert std to variance and create diagonal covariance
        
        # 3. Construct the updated Multivariate Normal distribution
        new_posterior = MultivariateNormal(mean_pred, updated_cov)
        return new_posterior

    def sample(self, distribution):
        # Sample self.num_samples realization(s) from the adjusted GP
        new_sample = distribution.rsample(torch.Size([self.num_samples]))  # Shape: (num_samples, test_x.size(0))
        return new_sample

    def visualize_corr(self):
        # Get test data
        test_x, _ = self.get_batch(self.test_loader, first=True)
        
        # Compute kernel matrix
        kernel_matrix = self.gp_model.covar_module(test_x).evaluate()
        
        # Visualize correlation
        plt.figure(figsize=(8, 6))
        sns.heatmap(kernel_matrix.detach().to('cpu'), cmap="coolwarm", square=True, cbar=True)
        plt.title("Kernel Matrix Correlation")
        plt.show()

    def visualize_gp(self):
        self.gp_model.eval()
        self.gp_model.likelihood.eval()
        
        # Check if BoTorch model is standardizing outputs
        if hasattr(self.gp_model, 'outcome_transform'):
            print("Model has outcome transform!")
            # Untransform predictions if there is a transform
            outcome_transform = self.gp_model.outcome_transform
        else:
            outcome_transform = None
        
        # Get predictions
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            y_pred_dist = self.gp_model(self.gp_train_x)
            mean = y_pred_dist.mean
            lower, upper = y_pred_dist.confidence_region()
            
            # If there is an outcome transform, untransform the predictions
            if outcome_transform:
                mean, _ = outcome_transform.untransform(mean)
                lower, _ = outcome_transform.untransform(lower)
                upper, _ = outcome_transform.untransform(upper)
        
        # Reshape self.gp_train_x to match the shape of mean
        self.gp_train_x = self.gp_train_x.flatten()  # Flatten the x-values
        
        # Plot results
        plt.figure(figsize=(10, 6))
        plt.plot(self.gp_train_x.cpu(), mean.cpu().flatten(), 'k-', label='Predicted mean')  # Flatten the mean
        plt.plot(self.gp_train_x.cpu(), self.gp_train_y.cpu(), color='blue', label='Training data')
        plt.fill_between(self.gp_train_x.cpu().flatten(), 
                         lower.cpu().flatten(), 
                         upper.cpu().flatten(), 
                         color='gray', alpha=0.3, 
                         label='95% confidence interval')
        plt.legend()
        plt.show()

    def visualize_sample(self, num_samples=3, on='test'):
        # 1. plot the model predictions
        x, y = self.get_batch(self.test_loader, first=True)
        for series in range(7):
            window = 0
            x, y = self.get_batch(self.test_loader, first=True, window=window, series=series)
            mean_pred = self.predictions[window, :, series, 0]  # Shape: [num_time_steps]
            std_pred = self.predictions[window, :, series, 1]   # Shape: [num_time_steps]
            
            # 4. define new posterior
            lower_bound = mean_pred + 2*std_pred
            upper_bound = mean_pred - 2*std_pred 
    
            plt.figure(figsize=(10, 6))
            plt.plot(x.cpu().flatten(), y.cpu().flatten(), 'k-', label='True Values')  # Flatten the mean
            plt.plot(x[-len(mean_pred):].cpu().flatten(), mean_pred.cpu(), color='red', label="Median Prediction")
            plt.fill_between(x[-len(mean_pred):].cpu().flatten(), lower_bound.cpu(), upper_bound.cpu(), color='red', alpha=0.3)
            # Plot results
            plt.legend()
            plt.show()


        new_posterior = self.init_distribution(mean_pred, std_pred)
        sample = self.sample(new_posterior)
        self.gp_model.eval()
        self.gp_model.likelihood.eval()
        
        # Check if BoTorch model is standardizing outputs
        if hasattr(self.gp_model, 'outcome_transform'):
            print("Model has outcome transform!")
            # Untransform predictions if there is a transform
            outcome_transform = self.gp_model.outcome_transform
        else:
            outcome_transform = None
        
        # Get predictions
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            y_pred_dist = self.gp_model(self.gp_train_x)
            mean = y_pred_dist.mean
            lower, upper = y_pred_dist.confidence_region()
            
            # If there is an outcome transform, untransform the predictions
            if outcome_transform:
                mean, _ = outcome_transform.untransform(mean)
                lower, _ = outcome_transform.untransform(lower)
                upper, _ = outcome_transform.untransform(upper)
        
        # Reshape self.gp_train_x to match the shape of mean
        self.gp_train_x = self.gp_train_x.flatten()  # Flatten the x-values
        
        # Plot results
        plt.figure(figsize=(10, 6))
        plt.plot(self.gp_train_x.cpu(), mean.cpu().flatten(), 'k-', label='Predicted mean')  # Flatten the mean
        plt.plot(self.gp_train_x.cpu(), self.gp_train_y.cpu(), color='blue', label='Training data')
        plt.fill_between(self.gp_train_x.cpu().flatten(), 
                         lower.cpu().flatten(), 
                         upper.cpu().flatten(), 
                         color='gray', alpha=0.3, 
                         label='95% confidence interval')
        plt.legend()
        plt.show()
        
        
from botorch.models import SingleTaskGP
gp_model = SingleTaskGP
sampl = SeriesSampler_1(gp_model, runner, num_samples=1)
sampl.run()
#sampl.visualize_gp()
sampl.visualize_sample()

In [ ]:
def get_batch(data_loader, first=True, window=0, series=0):
        if first:
            # select the first batch
            _iter = iter(data_loader) 
            batch = next(_iter)
        else:
            # select the last batch
            num_batches = len(data_loader)
            if num_batches == 0:
                return None, None
            _iter = iter(data_loader)
            for i in range(num_batches - 1):
                next(_iter)
            batch = next(_iter)

        history = batch['inputs'][..., :1]     # bs x length x num_series x features [33, 336, 7, 1]
        target = batch['target'][..., :1]     # bs x length x num_series x features [33, 336, 7, 1]
        targets = torch.cat([history, target], dim=1)  # batch_size, horizon, num_series = (32, 360, 7)
        targets = targets[window, :, series]  # only select first batch/window at first (360, num_series)
        horizon_length = targets.shape[0]
    
        x = torch.arange(horizon_length).to('cuda', dtype=torch.float32).unsqueeze(-1)  # [360, 1]
        y = targets.to('cuda', dtype=torch.float32)  # [360, num_series, 1]
        return x, y
    
def get_predictions_base_model(on='val', runner=None):
    runner.model.eval()  # Ensure model is in eval mode
    runner.model.to('cuda')  # Move to appropriate device
    predictions = []
    if on == 'val':
        dataloader = self.val_loader
    else: 
        dataloader = runner.test_data_loader
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader)):
            # TODO scaler 
            model_return = runner.forward(batch, epoch=None, iter_num=None, train=False)
            if runner.scaler:
                model_return['prediction'] = runner.scaler.inverse_transform(model_return['prediction'])
                model_return['target'] = runner.scaler.inverse_transform(model_return['target'])
                model_return['inputs'] = runner.scaler.inverse_transform(model_return['inputs'])
            #model_return = self.runner.postprocessing(model_return)
            #forecasts = model(batch['inputs'][..., :1].to(device), batch['target'].to(device), batch_seen=0, epoch=0, train=False)
            forecasts = model_return['prediction']
            predictions.append(forecasts)
    predictions = torch.cat(predictions, dim=0)
    return predictions

runner = init(model='PatchTST', dataset='ETTh1', no_logging=True)
predictions = get_predictions_base_model(on='test', runner=runner)
for series in range(7):
    window = 0
    x, y = get_batch(runner.test_data_loader, first=True, window=window, series=series)
    pred = predictions[window, :, series]  # Shape: [num_time_steps]
    
    plt.figure(figsize=(10, 6))
    plt.plot(x.cpu().flatten(), y.cpu().flatten(), 'k-', label='True Values')  # Flatten the mean
    plt.plot(x[-len(pred):].cpu().flatten(), pred.cpu(), color='red', label="Median Prediction")
    plt.legend()
    plt.show()